# Join Data Banjir dengan Data Kecamatan (Enhanced)

Notebook ini akan melakukan join antara:
- `data_banjir_combine_final.csv` (18,047 baris) 
- `kecamatan_filtered.csv` (7,287 baris)

## ✨ Fitur Perbaikan:
1. **Fallback Strategy**: Gunakan NAME_3 jika NAME_3_clean kosong
2. **Fuzzy Matching**: Matching dengan similarity 85%+ untuk kecamatan yang hampir sama
3. **Smart Deduplication**: Pilih record dengan populasi terbesar untuk duplikat
4. **Comprehensive Validation**: Analisis detail hasil join dan data yang tidak match

## 🎯 Target:
- Mempertahankan semua 18,047 records dari data banjir (left join)
- Maksimalkan matching rate dengan data kecamatan
- Memperkaya data banjir dengan informasi demografi dan ekonomi dari data kecamatan

In [48]:
import pandas as pd
import numpy as np
import re
from unicodedata import normalize
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Data

In [49]:
df_banjir = pd.read_csv('data_banjir_combine_final.csv')
df_kecamatan = pd.read_csv('kecamatan_filtered.csv')

print(df_banjir.head(3))

print(df_kecamatan.head(3))

print(df_kecamatan.columns.tolist())

  NAME_2         NAME_3  avg_rainfall  max_rainfall  avg_temperature  \
0  bogor  babakanmadang        64.775       124.032            28.76   
1  bogor     bojonggede        68.241       128.949            32.89   
2  bogor       caringin        56.423        92.324            29.93   

   elevation landcover_class   ndvi  slope  soil_moisture  year  month  \
0    501.913      Tree cover  0.564  5.001         39.583  2020      1   
1    133.589        Built-up  0.429  0.293         40.414  2020      1   
2    765.651      Tree cover  0.513  4.954         39.662  2020      1   

   banjir     lat      long                  map_image   NAME_3_clean  
0       1 -6.5917  106.9020  RGB_BabakanMadang2020.tif  babakanmadang  
1       1 -6.4774  106.7927  RGB_BabakanMadang2020.tif     bojonggede  
2       0 -6.7366  106.8660    RGB_Caringin2020(2).tif       caringin  
   objectid  kode_kec_spatial            nama_prop         nama_kab  \
0       NaN          130515.0       SUMATERA BARAT  PAD

## 2. Fungsi Normalisasi Nama Kecamatan

Membuat fungsi untuk menormalisasi nama kecamatan agar matching lebih akurat:
- Convert ke lowercase
- Hapus spasi berlebih
- Hapus karakter khusus
- Normalisasi unicode (untuk karakter Indonesia)

In [61]:
def normalize_kecamatan_name(name):
    """
    Normalisasi nama kecamatan untuk matching yang lebih baik
    
    Args:
        name: nama kecamatan (string atau NaN)
    
    Returns:
        normalized string atau empty string jika NaN
    """
    if pd.isna(name):
        return ''
    
    name = str(name)
    name = normalize('NFD', name).encode('ascii', 'ignore').decode('utf-8')
    name = name.lower()
    name = re.sub(r'[^a-z0-9\s]', '', name)
    name = re.sub(r'\s+', '', name)
    
    return name.strip()

test_names = [
    'Babakan Madang',
    'BOJONGGEDE',
    'Cibinong',
    'Kota Tangerang',
    '2 X 11 KAYU TANAM',
    None,
    ''
]

for test in test_names:
    print(f"{str(test):30s} -> {normalize_kecamatan_name(test)}")

Babakan Madang                 -> babakanmadang
BOJONGGEDE                     -> bojonggede
Cibinong                       -> cibinong
Kota Tangerang                 -> kotatangerang
2 X 11 KAYU TANAM              -> 2x11kayutanam
None                           -> 
                               -> 


## 3. Apply Normalisasi ke Kedua Dataset

In [52]:
# Strategi join yang lebih baik: gunakan NAME_3 sebagai fallback jika NAME_3_clean kosong
df_banjir['kecamatan_for_join'] = df_banjir['NAME_3_clean'].fillna(df_banjir['NAME_3'])
df_banjir['kecamatan_key'] = df_banjir['kecamatan_for_join'].apply(normalize_kecamatan_name)

df_kecamatan['kecamatan_key'] = df_kecamatan['nama_kec'].apply(normalize_kecamatan_name)

print("=== Status setelah perbaikan ===")
print(f"Empty keys in df_banjir: {(df_banjir['kecamatan_key'] == '').sum()}")
print(f"Empty keys in df_kecamatan: {(df_kecamatan['kecamatan_key'] == '').sum()}")

print("\n=== Sample df_banjir ===")
print(df_banjir[['NAME_2', 'NAME_3', 'NAME_3_clean', 'kecamatan_for_join', 'kecamatan_key']].head(10))
print("\n=== Sample df_kecamatan ===")
print(df_kecamatan[['nama_kec', 'kecamatan_key']].head(10))

=== Status setelah perbaikan ===
Empty keys in df_banjir: 0
Empty keys in df_kecamatan: 39

=== Sample df_banjir ===
  NAME_2         NAME_3   NAME_3_clean kecamatan_for_join  kecamatan_key
0  bogor  babakanmadang  babakanmadang      babakanmadang  babakanmadang
1  bogor     bojonggede     bojonggede         bojonggede     bojonggede
2  bogor       caringin       caringin           caringin       caringin
3  bogor          cariu          cariu              cariu          cariu
4  bogor        ciampea        ciampea            ciampea        ciampea
5  bogor          ciawi          ciawi              ciawi          ciawi
6  bogor       cibinong       cibinong           cibinong       cibinong
7  bogor   cibungbulang   cibungbulang       cibungbulang   cibungbulang
8  bogor      cigombong      cigombong          cigombong      cigombong
9  bogor        cigudeg        cigudeg            cigudeg        cigudeg

=== Sample df_kecamatan ===
            nama_kec  kecamatan_key
0  2 X 11 KAYU 

In [51]:
# Investigasi data kosong di df_banjir
print("Investigasi NAME_3_clean yang menghasilkan kecamatan_key kosong:")
empty_key_banjir = df_banjir[df_banjir['kecamatan_key'] == '']
print(f"\nJumlah: {len(empty_key_banjir)}")
print("\nSample data dengan key kosong:")
print(empty_key_banjir[['NAME_2', 'NAME_3', 'NAME_3_clean', 'kecamatan_key']].head(20))

# Cek apakah NAME_3_clean memang kosong
print(f"\nNAME_3_clean yang NULL/NaN: {df_banjir['NAME_3_clean'].isna().sum()}")
print(f"NAME_3_clean yang kosong (''): {(df_banjir['NAME_3_clean'] == '').sum()}")
print(f"NAME_3 yang NULL/NaN: {df_banjir['NAME_3'].isna().sum()}")
print(f"NAME_3 yang kosong (''): {(df_banjir['NAME_3'] == '').sum()}")

Investigasi NAME_3_clean yang menghasilkan kecamatan_key kosong:

Jumlah: 7067

Sample data dengan key kosong:
              NAME_2          NAME_3 NAME_3_clean kecamatan_key
10980   Aceh Selatan     kluettengah          NaN              
10981   Aceh Selatan      kluettimur          NaN              
10982   Aceh Selatan          meukek          NaN              
10983   Aceh Selatan         samadua          NaN              
10984   Aceh Selatan       tapaktuan          NaN              
10985   Aceh Selatan          trumon          NaN              
10986   Aceh Singkil    simpangkanan          NaN              
10987           Agam     kamangmagek          NaN              
10988           Agam     lubukbasung          NaN              
10989           Agam      palembayan          NaN              
10990           Agam  tanjungmutiara          NaN              
10991        Bandung       baleendah          NaN              
10992        Bandung     bojongsoang          NaN        

## 4. Analisis Overlap Sebelum Join

Cek berapa banyak kecamatan yang ada di kedua dataset

In [53]:
unique_banjir = set(df_banjir['kecamatan_key'].unique())
unique_kecamatan = set(df_kecamatan['kecamatan_key'].unique())

unique_banjir.discard('')
unique_kecamatan.discard('')

overlap = unique_banjir.intersection(unique_kecamatan)
only_in_banjir = unique_banjir - unique_kecamatan
only_in_kecamatan = unique_kecamatan - unique_banjir

print("=" * 70)
print("ANALISIS OVERLAP KECAMATAN")
print("=" * 70)
print(f"Unique kecamatan di df_banjir: {len(unique_banjir)}")
print(f"Unique kecamatan di df_kecamatan: {len(unique_kecamatan)}")
print(f"Kecamatan yang ada di KEDUA dataset: {len(overlap)} ({len(overlap)/len(unique_banjir)*100:.1f}%)")
print(f"Kecamatan HANYA di df_banjir: {len(only_in_banjir)} ({len(only_in_banjir)/len(unique_banjir)*100:.1f}%)")
print(f"Kecamatan HANYA di df_kecamatan: {len(only_in_kecamatan)}")

print(f"\n{'='*70}")
print("SAMPLE KECAMATAN YANG MATCH (20 pertama)")
print("=" * 70)
for i, kec in enumerate(sorted(list(overlap))[:20], 1):
    print(f"{i:3d}. {kec}")

if len(only_in_banjir) > 0:
    print(f"\n{'='*70}")
    print("SAMPLE KECAMATAN TIDAK MATCH - Hanya di df_banjir (10 pertama)")
    print("=" * 70)
    for i, kec in enumerate(sorted(list(only_in_banjir))[:10], 1):
        original = df_banjir[df_banjir['kecamatan_key'] == kec]['kecamatan_for_join'].iloc[0]
        print(f"{i:3d}. {original:30s} -> {kec}")

ANALISIS OVERLAP KECAMATAN
Unique kecamatan di df_banjir: 2747
Unique kecamatan di df_kecamatan: 6846
Kecamatan yang ada di KEDUA dataset: 2715 (98.8%)
Kecamatan HANYA di df_banjir: 32 (1.2%)
Kecamatan HANYA di df_kecamatan: 4131

SAMPLE KECAMATAN YANG MATCH (20 pertama)
  1. abang
  2. abuki
  3. adimulyo
  4. adipala
  5. adiwerna
  6. adonara
  7. adonarabarat
  8. adonaratimur
  9. aeksongsongan
 10. aesesa
 11. agrabinta
 12. airbatu
 13. airbesar
 14. airbesi
 15. airhangat
 16. airhangattimur
 17. airjoman
 18. airmadidi
 19. airnaningan
 20. airnapal

SAMPLE KECAMATAN TIDAK MATCH - Hanya di df_banjir (10 pertama)
  1. alu                            -> alu
  2. argamakmur                     -> argamakmur
  3. balabalakang                   -> balabalakang
  4. batanggansal                   -> batanggansal
  5. bupon                          -> bupon
  6. canduang                       -> canduang
  7. cigemblong                     -> cigemblong
  8. duakoto                   

## 5. Perform Join

Gunakan **left join** untuk mempertahankan semua data dari df_banjir dan tambahkan informasi dari df_kecamatan jika ada.

### 4.5 Fuzzy Matching untuk Kecamatan yang Tidak Match

Untuk meningkatkan matching, kita gunakan fuzzy matching pada kecamatan yang tidak match persis.

In [54]:
from difflib import SequenceMatcher

def find_best_match(key, candidates, threshold=0.85):
    """
    Cari match terbaik menggunakan fuzzy matching
    
    Args:
        key: string yang akan dicari
        candidates: set/list kandidat
        threshold: minimum similarity ratio (0-1)
    
    Returns:
        best_match atau None jika tidak ada yang memenuhi threshold
    """
    if not key or key == '':
        return None
    
    best_match = None
    best_ratio = 0
    
    for candidate in candidates:
        if not candidate or candidate == '':
            continue
        ratio = SequenceMatcher(None, key, candidate).ratio()
        if ratio > best_ratio and ratio >= threshold:
            best_ratio = ratio
            best_match = candidate
    
    return best_match if best_ratio >= threshold else None

# Buat mapping untuk kecamatan yang tidak match
only_in_banjir_list = sorted(list(only_in_banjir))
kecamatan_mapping = {}

print("Mencari fuzzy match untuk kecamatan yang tidak match persis...")
print("=" * 80)

for kec in only_in_banjir_list:
    match = find_best_match(kec, unique_kecamatan, threshold=0.85)
    if match:
        original = df_banjir[df_banjir['kecamatan_key'] == kec]['kecamatan_for_join'].iloc[0]
        matched_original = df_kecamatan[df_kecamatan['kecamatan_key'] == match]['nama_kec'].iloc[0]
        ratio = SequenceMatcher(None, kec, match).ratio()
        kecamatan_mapping[kec] = match
        print(f"{original:25s} -> {matched_original:25s} (similarity: {ratio:.2%})")

print(f"\n{'='*80}")
print(f"Total fuzzy match ditemukan: {len(kecamatan_mapping)} dari {len(only_in_banjir_list)}")
print(f"Masih tidak match: {len(only_in_banjir_list) - len(kecamatan_mapping)}")

# Apply mapping
if len(kecamatan_mapping) > 0:
    df_banjir['kecamatan_key_enhanced'] = df_banjir['kecamatan_key'].replace(kecamatan_mapping)
    print(f"\nMapping diterapkan ke df_banjir sebagai kolom 'kecamatan_key_enhanced'")
else:
    df_banjir['kecamatan_key_enhanced'] = df_banjir['kecamatan_key']
    print("\nTidak ada fuzzy match yang ditemukan, menggunakan key original")

Mencari fuzzy match untuk kecamatan yang tidak match persis...
alu                       -> ALLU                      (similarity: 85.71%)
balabalakang              -> KEP. BALA BALAKANG        (similarity: 88.89%)
batanggansal              -> BATANG GANGSAL            (similarity: 96.00%)
canduang                  -> CANDUNG                   (similarity: 93.33%)
cigemblong                -> CIGEMLONG                 (similarity: 94.74%)
duakoto                   -> DUO KOTO                  (similarity: 85.71%)
indramakmur               -> INDRA MAKMU               (similarity: 95.24%)
ixkoto                    -> X KOTO                    (similarity: 90.91%)
kelapanunggal             -> KLAPANUNGGAL              (similarity: 96.00%)
klubagolit                -> KELUBAGOLIT               (similarity: 95.24%)
lubuktarok                -> LUBUAK TAROK              (similarity: 95.24%)
luttawar                  -> LAUT TAWAR                (similarity: 94.12%)
pecangan                 

In [55]:
print("=" * 80)
print("CEK DUPLIKASI SEBELUM JOIN")
print("=" * 80)
print(f"Duplikasi di df_banjir (kecamatan_key_enhanced): {df_banjir['kecamatan_key_enhanced'].duplicated().sum()}")
print(f"Duplikasi di df_kecamatan (kecamatan_key): {df_kecamatan['kecamatan_key'].duplicated().sum()}")

if df_kecamatan['kecamatan_key'].duplicated().sum() > 0:
    print("\n⚠️  Ditemukan duplikasi di df_kecamatan!")
    print("\nSample duplikat kecamatan:")
    dupes = df_kecamatan[df_kecamatan['kecamatan_key'].duplicated(keep=False)].sort_values('kecamatan_key')
    print(dupes[['nama_prop', 'nama_kab', 'nama_kec', 'kecamatan_key']].head(20))
    
    # Agregasi data duplikat: ambil yang jumlah_penduduk terbesar
    print("\n📊 Strategi penanganan duplikasi:")
    print("   - Untuk setiap kecamatan_key yang duplikat, pilih yang jumlah_penduduk terbesar")
    print("   - Atau jika jumlah_penduduk sama/kosong, pilih yang pertama muncul")
    
    df_kecamatan_clean = (df_kecamatan
                          .sort_values('jumlah_penduduk', ascending=False, na_position='last')
                          .drop_duplicates(subset='kecamatan_key', keep='first')
                          .reset_index(drop=True))
    
    print(f"\n✅ Setelah deduplikasi: {len(df_kecamatan)} -> {len(df_kecamatan_clean)} rows")
    print(f"   Dihapus: {len(df_kecamatan) - len(df_kecamatan_clean)} duplikat")
else:
    df_kecamatan_clean = df_kecamatan.copy()
    print("\n✅ Tidak ada duplikasi di df_kecamatan")

CEK DUPLIKASI SEBELUM JOIN
Duplikasi di df_banjir (kecamatan_key_enhanced): 15302
Duplikasi di df_kecamatan (kecamatan_key): 438

⚠️  Ditemukan duplikasi di df_kecamatan!

Sample duplikat kecamatan:
     nama_prop nama_kab nama_kec kecamatan_key
18         NaN      NaN      NaN              
698        NaN      NaN      NaN              
1084       NaN      NaN      NaN              
1694       NaN      NaN      NaN              
1876       NaN      NaN      NaN              
2032       NaN      NaN      NaN              
5261       NaN      NaN      NaN              
2371       NaN      NaN      NaN              
2403       NaN      NaN      NaN              
2404       NaN      NaN      NaN              
2405       NaN      NaN      NaN              
622        NaN      NaN      NaN              
2424       NaN      NaN      NaN              
3003       NaN      NaN      NaN              
3035       NaN      NaN      NaN              
4256       NaN      NaN      NaN              
42

In [56]:
print("=" * 80)
print("MELAKUKAN JOIN DATASET")
print("=" * 80)

# Join menggunakan kecamatan_key_enhanced
df_joined = df_banjir.merge(
    df_kecamatan_clean,
    left_on='kecamatan_key_enhanced',
    right_on='kecamatan_key',
    how='left',
    suffixes=('_banjir', '_kecamatan')
)

print(f"\n✅ JOIN BERHASIL!")
print(f"   Shape df_banjir (original): {df_banjir.shape}")
print(f"   Shape df_kecamatan (cleaned): {df_kecamatan_clean.shape}")
print(f"   Shape df_joined (result): {df_joined.shape}")

print(f"\n📊 Sample hasil join:")
print(df_joined[['NAME_2', 'kecamatan_for_join', 'nama_kab', 'nama_kec', 'jumlah_penduduk']].head(10))

MELAKUKAN JOIN DATASET

✅ JOIN BERHASIL!
   Shape df_banjir (original): (18047, 20)
   Shape df_kecamatan (cleaned): (6847, 22)
   Shape df_joined (result): (18047, 42)

📊 Sample hasil join:
  NAME_2 kecamatan_for_join nama_kab        nama_kec  jumlah_penduduk
0  bogor      babakanmadang    BOGOR  BABAKAN MADANG         121972.0
1  bogor         bojonggede    BOGOR     BOJONG GEDE         266481.0
2  bogor           caringin    BOGOR        CARINGIN         138901.0
3  bogor              cariu    BOGOR           CARIU          54478.0
4  bogor            ciampea    BOGOR         CIAMPEA         175400.0
5  bogor              ciawi    BOGOR           CIAWI         117903.0
6  bogor           cibinong    BOGOR        CIBINONG         365387.0
7  bogor       cibungbulang    BOGOR    CIBUNGBULANG         152898.0
8  bogor          cigombong    BOGOR       CIGOMBONG         101474.0
9  bogor            cigudeg    BOGOR         CIGUDEG         142405.0


## 6. Validasi Hasil Join

In [57]:
print("=" * 80)
print("VALIDASI HASIL JOIN")
print("=" * 80)

if 'nama_kab' in df_joined.columns:
    matched_rows = df_joined['nama_kab'].notna().sum()
    unmatched_rows = df_joined['nama_kab'].isna().sum()
    
    print(f"\n📊 STATISTIK JOIN:")
    print(f"   Total baris: {len(df_joined):,}")
    print(f"   ✅ Baris yang berhasil di-join: {matched_rows:,} ({matched_rows/len(df_joined)*100:.2f}%)")
    print(f"   ❌ Baris yang TIDAK ter-join: {unmatched_rows:,} ({unmatched_rows/len(df_joined)*100:.2f}%)")
    
    if unmatched_rows > 0:
        print(f"\n{'='*80}")
        print(f"SAMPLE DATA YANG TIDAK TER-JOIN ({min(10, unmatched_rows)} dari {unmatched_rows})")
        print("=" * 80)
        unmatched = df_joined[df_joined['nama_kab'].isna()]
        unmatched_unique = unmatched[['NAME_2', 'NAME_3', 'kecamatan_for_join', 'kecamatan_key_enhanced']].drop_duplicates()
        print(unmatched_unique.head(10).to_string(index=False))
        
        # Analisis penyebab tidak match
        print(f"\n{'='*80}")
        print("ANALISIS PENYEBAB TIDAK MATCH:")
        print("=" * 80)
        unmatched_keys = set(unmatched['kecamatan_key_enhanced'].unique())
        unmatched_keys.discard('')
        print(f"   Jumlah unique kecamatan yang tidak match: {len(unmatched_keys)}")
        
        if len(unmatched_keys) > 0 and len(unmatched_keys) <= 20:
            print(f"\n   Daftar kecamatan yang tidak match:")
            for i, key in enumerate(sorted(unmatched_keys), 1):
                original = unmatched[unmatched['kecamatan_key_enhanced'] == key]['kecamatan_for_join'].iloc[0]
                count = len(unmatched[unmatched['kecamatan_key_enhanced'] == key])
                print(f"   {i:2d}. {original:30s} ({count:,} records)")
else:
    print("⚠️  Kolom 'nama_kab' tidak ditemukan. Cek nama kolom!")

# Statistik kolom dari kecamatan
print(f"\n{'='*80}")
print("STATISTIK KOLOM DARI DATA KECAMATAN")
print("=" * 80)
kecamatan_cols = [col for col in df_joined.columns if col in df_kecamatan_clean.columns or '_kecamatan' in col]
print(f"Jumlah kolom dari data kecamatan: {len(kecamatan_cols)}")

print(f"\n{'Kolom':<40s} {'Missing':<10s} {'Pct':<10s}")
print("-" * 60)
for col in kecamatan_cols[:15]:
    missing = df_joined[col].isna().sum()
    print(f"{col:<40s} {missing:>8,d}   {missing/len(df_joined)*100:>5.2f}%")

VALIDASI HASIL JOIN

📊 STATISTIK JOIN:
   Total baris: 18,047
   ✅ Baris yang berhasil di-join: 18,040 (99.96%)
   ❌ Baris yang TIDAK ter-join: 7 (0.04%)

SAMPLE DATA YANG TIDAK TER-JOIN (7 dari 7)
        NAME_2         NAME_3 kecamatan_for_join kecamatan_key_enhanced
   Buton Utara      wakorumba          wakorumba              wakorumba
          Luwu          bupon              bupon                  bupon
        Majene           tubo               tubo                   tubo
        Majene      tammerodo          tammerodo              tammerodo
  Lombok Barat sekotongtengah     sekotongtengah         sekotongtengah
Bengkulu Utara     argamakmur         argamakmur             argamakmur

ANALISIS PENYEBAB TIDAK MATCH:
   Jumlah unique kecamatan yang tidak match: 6

   Daftar kecamatan yang tidak match:
    1. argamakmur                     (1 records)
    2. bupon                          (1 records)
    3. sekotongtengah                 (1 records)
    4. tammerodo              

## 7. Clean Up & Export

Hapus kolom helper (kecamatan_key) dan export hasil join

In [58]:
print("=" * 80)
print("CLEANUP & PERSIAPAN EXPORT")
print("=" * 80)

df_final = df_joined.copy()

# Hapus kolom-kolom helper/temporary
cols_to_drop = ['kecamatan_key', 'kecamatan_key_enhanced', 'kecamatan_for_join']
existing_cols_to_drop = [col for col in cols_to_drop if col in df_final.columns]

if existing_cols_to_drop:
    df_final = df_final.drop(columns=existing_cols_to_drop)
    print(f"\n✅ Kolom helper dihapus: {', '.join(existing_cols_to_drop)}")

print(f"\n📊 INFORMASI DATASET FINAL:")
print(f"   Shape: {df_final.shape}")
print(f"   Total rows: {len(df_final):,}")
print(f"   Total columns: {len(df_final.columns)}")

print(f"\n{'='*80}")
print("DAFTAR KOLOM (50 pertama):")
print("=" * 80)
for i, col in enumerate(df_final.columns[:50], 1):
    missing = df_final[col].isna().sum()
    print(f"{i:3d}. {col:<35s} (missing: {missing:>6,d} / {missing/len(df_final)*100:>5.1f}%)")

if len(df_final.columns) > 50:
    print(f"... dan {len(df_final.columns) - 50} kolom lainnya")

print(f"\n📈 Sample data:")
print(df_final.head(3))

CLEANUP & PERSIAPAN EXPORT

✅ Kolom helper dihapus: kecamatan_key_enhanced, kecamatan_for_join

📊 INFORMASI DATASET FINAL:
   Shape: (18047, 40)
   Total rows: 18,047
   Total columns: 40

DAFTAR KOLOM (50 pertama):
  1. NAME_2                              (missing:      0 /   0.0%)
  2. NAME_3                              (missing:      0 /   0.0%)
  3. avg_rainfall                        (missing:      0 /   0.0%)
  4. max_rainfall                        (missing:      0 /   0.0%)
  5. avg_temperature                     (missing:      0 /   0.0%)
  6. elevation                           (missing:      0 /   0.0%)
  7. landcover_class                     (missing:      0 /   0.0%)
  8. ndvi                                (missing:      0 /   0.0%)
  9. slope                               (missing:      0 /   0.0%)
 10. soil_moisture                       (missing:      0 /   0.0%)
 11. year                                (missing:      0 /   0.0%)
 12. month                          

In [59]:
output_filename = 'data_banjir_joined.csv'

try:
    df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')
    print(f"Data berhasil disimpan ke: {output_filename}")
    print(f"  - Total rows: {len(df_final):,}")
    print(f"  - Total columns: {len(df_final.columns)}")
    import os
    file_size = os.path.getsize(output_filename) / (1024 * 1024)
    print(f"  - File size: {file_size:.2f} MB")
    
except Exception as e:
    print(f"Error saat menyimpan file: {e}")

Data berhasil disimpan ke: data_banjir_joined.csv
  - Total rows: 18,047
  - Total columns: 40
  - File size: 5.01 MB


In [40]:
df_joined_clean = pd.read_csv('data_banjir_joined.csv')

print(f"Shape: {df_joined_clean.shape}")
print(f"\nKolom yang akan diubah:")
print(f"  - NAME_2: {df_joined_clean['NAME_2'].iloc[0]}")
print(f"  - NAME_3: {df_joined_clean['NAME_3'].iloc[0]}")
print(f"  - NAME_3_clean: {df_joined_clean['NAME_3_clean'].iloc[0]}")

if 'objectid' in df_joined_clean.columns:
    null_count = df_joined_clean['objectid'].isna().sum()
    total_count = len(df_joined_clean)
    print(f"\n  - objectid: {null_count}/{total_count} kosong ({null_count/total_count*100:.1f}%)")
else:
    print("\n  - objectid: kolom tidak ditemukan")

Shape: (18047, 38)

Kolom yang akan diubah:
  - NAME_2: bogor
  - NAME_3: babakanmadang
  - NAME_3_clean: babakanmadang

  - objectid: 18047/18047 kosong (100.0%)


C:\Users\admin\AppData\Local\Temp\ipykernel_2992\1221362077.py:1: DtypeWarning: Columns (16,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_joined_clean = pd.read_csv('data_banjir_joined.csv')


In [41]:
rename_dict = {
    'NAME_2': 'kabupaten_kota',      
    'NAME_3': 'kecamatan',            
    'NAME_3_clean': 'kecamatan_clean' 
}

df_joined_clean = df_joined_clean.rename(columns=rename_dict)

for old_name, new_name in rename_dict.items():
    print(f"  {old_name:20s} -> {new_name}")

if 'objectid' in df_joined_clean.columns:
    df_joined_clean = df_joined_clean.drop(columns=['objectid'])
    print("\n  Kolom 'objectid' telah dihapus")
else:
    print("\n  Kolom 'objectid' tidak ditemukan")


print(f"Shape: {df_joined_clean.shape}")
print(f"\nSample data dengan nama kolom baru:")
print(df_joined_clean[['kabupaten_kota', 'kecamatan', 'kecamatan_clean']].head())

  NAME_2               -> kabupaten_kota
  NAME_3               -> kecamatan
  NAME_3_clean         -> kecamatan_clean

  Kolom 'objectid' telah dihapus
Shape: (18047, 37)

Sample data dengan nama kolom baru:
  kabupaten_kota      kecamatan kecamatan_clean
0          bogor  babakanmadang   babakanmadang
1          bogor     bojonggede      bojonggede
2          bogor       caringin        caringin
3          bogor          cariu           cariu
4          bogor        ciampea         ciampea


In [42]:
for i, col in enumerate(df_joined_clean.columns, 1):
    if col in ['kabupaten_kota', 'kecamatan', 'kecamatan_clean']:
        print(f"{i:3d}. {col} *")
    else:
        print(f"{i:3d}. {col}")

print(f"\nTotal kolom: {len(df_joined_clean.columns)}")
print(f"Total baris: {len(df_joined_clean):,}")

  1. kabupaten_kota *
  2. kecamatan *
  3. avg_rainfall
  4. max_rainfall
  5. avg_temperature
  6. elevation
  7. landcover_class
  8. ndvi
  9. slope
 10. soil_moisture
 11. year
 12. month
 13. banjir
 14. lat
 15. long
 16. map_image
 17. kecamatan_clean *
 18. kode_kec_spatial
 19. nama_prop
 20. nama_kab
 21. nama_kec
 22. jumlah_penduduk
 23. kepadatan_penduduk
 24. jumlah_kk
 25. luas_wilayah
 26. pertumbuhan_2022
 27. pertumbuhan_2021
 28. pertumbuhan_2020
 29. perpindahan_pddk
 30. jumlah_desa
 31. jumlah_kelurahan
 32. perdagangan
 33. nelayan
 34. wiraswasta
 35. tidak_blm_sekolah
 36. slta
 37. s1

Total kolom: 37
Total baris: 18,047


## 9. Export Data Final yang Sudah Dibersihkan

In [43]:
output_filename_final = 'data_banjir_joined_clean.csv'

try:
    df_joined_clean.to_csv(output_filename_final, index=False, encoding='utf-8-sig')
    print(f"File: {output_filename_final}")
    print(f"Total rows: {len(df_joined_clean):,}")
    print(f"Total columns: {len(df_joined_clean.columns)}")
    
    import os
    file_size = os.path.getsize(output_filename_final) / (1024 * 1024)
    print(f"File size: {file_size:.2f} MB")
    
    print("  NAME_2 -> kabupaten_kota")
    print("  NAME_3 -> kecamatan")
    print("  NAME_3_clean -> kecamatan_clean")
    print("  Kolom 'objectid' dihapus")
    
except Exception as e:
    print(f"Error saat menyimpan file: {e}")

File: data_banjir_joined_clean.csv
Total rows: 18,047
Total columns: 37
File size: 3.87 MB
  NAME_2 -> kabupaten_kota
  NAME_3 -> kecamatan
  NAME_3_clean -> kecamatan_clean
  Kolom 'objectid' dihapus


In [44]:
print(df_joined_clean.head(10))

print(df_joined_clean.info())

  kabupaten_kota      kecamatan  avg_rainfall  max_rainfall  avg_temperature  \
0          bogor  babakanmadang        64.775       124.032            28.76   
1          bogor     bojonggede        68.241       128.949            32.89   
2          bogor       caringin        56.423        92.324            29.93   
3          bogor          cariu        60.034        98.129            30.02   
4          bogor        ciampea        66.380       133.877            30.27   
5          bogor          ciawi        60.033       102.824            30.31   
6          bogor       cibinong        64.197       120.134            32.59   
7          bogor   cibungbulang        62.757       125.628            30.10   
8          bogor      cigombong        53.486        84.206            27.78   
9          bogor        cigudeg        61.410       109.292            28.26   

   elevation landcover_class   ndvi  slope  soil_moisture  year  month  \
0    501.913      Tree cover  0.564  5.001   

## 🎯 RINGKASAN PERBAIKAN JOIN DATASET

### Masalah yang Ditemukan:
1. **7,067 baris** memiliki `NAME_3_clean` yang kosong (NaN) → menyebabkan kecamatan_key kosong
2. Duplikasi di df_kecamatan (438 baris duplikat)
3. Matching rate rendah karena tidak ada fallback strategy

### Solusi yang Diterapkan:

#### 1. **Fallback Strategy untuk Kecamatan Key**
   - Gunakan `NAME_3_clean` jika tersedia
   - Fallback ke `NAME_3` jika `NAME_3_clean` kosong
   - Hasil: **0 empty keys** (sebelumnya: 7,067)

#### 2. **Fuzzy Matching untuk Kecamatan yang Tidak Match Persis**
   - Menggunakan `SequenceMatcher` dengan threshold 85%
   - Menemukan similarity antara kecamatan yang hampir sama
   - Hasil: beberapa kecamatan tambahan berhasil di-match

#### 3. **Penanganan Duplikasi yang Lebih Baik**
   - Sort berdasarkan `jumlah_penduduk` (descending)
   - Pilih record dengan populasi terbesar untuk setiap kecamatan
   - Hasil: 7,285 → 6,847 rows (438 duplikat dihapus)

### Hasil Akhir:
- ✅ **Matching rate: ~99%** (meningkat signifikan dari sebelumnya)
- ✅ **18,047 rows** dipertahankan dari dataset banjir
- ✅ **40 kolom total** (19 dari data banjir + 21 dari data kecamatan)
- ✅ Data yang tidak match: minimal (hanya kecamatan yang benar-benar tidak ada di dataset kecamatan)

In [60]:
# PERBANDINGAN SEBELUM vs SESUDAH PERBAIKAN
print("=" * 80)
print("📊 PERBANDINGAN HASIL JOIN")
print("=" * 80)

# Hitung matching statistics
total_rows = len(df_final)
matched = df_final['nama_kab'].notna().sum()
unmatched = df_final['nama_kab'].isna().sum()

print(f"\n{'Metrik':<40s} {'Hasil':<15s} {'Persentase':<15s}")
print("-" * 70)
print(f"{'Total records di df_banjir':<40s} {total_rows:>12,d}")
print(f"{'Records berhasil di-join':<40s} {matched:>12,d}   {matched/total_rows*100:>10.2f}%")
print(f"{'Records tidak ter-join':<40s} {unmatched:>12,d}   {unmatched/total_rows*100:>10.2f}%")

print(f"\n{'='*80}")
print("🎯 PENCAPAIAN:")
print("=" * 80)
print("✅ Semua 18,047 records dari data banjir dipertahankan (left join)")
print(f"✅ {matched:,} records ({matched/total_rows*100:.1f}%) berhasil diperkaya dengan data kecamatan")
print(f"✅ Menambahkan ~21 kolom baru dari data kecamatan (demografi, ekonomi, dll)")

print(f"\n💡 Catatan:")
print(f"   Records yang tidak ter-join ({unmatched:,}) kemungkinan karena:")
print(f"   - Nama kecamatan tidak ada di dataset kecamatan_filtered.csv")
print(f"   - Perubahan nama kecamatan atau pemekaran wilayah")
print(f"   - Perbedaan tahun data (data banjir vs data kecamatan)")

# Tampilkan kolom-kolom baru yang ditambahkan
print(f"\n{'='*80}")
print("📋 KOLOM-KOLOM BARU DARI DATA KECAMATAN:")
print("=" * 80)
new_cols = [col for col in df_final.columns if col in df_kecamatan_clean.columns]
for i, col in enumerate(new_cols, 1):
    non_null = df_final[col].notna().sum()
    print(f"   {i:2d}. {col:<30s} ({non_null:>6,d} values / {non_null/total_rows*100:>5.1f}%)")

📊 PERBANDINGAN HASIL JOIN

Metrik                                   Hasil           Persentase     
----------------------------------------------------------------------
Total records di df_banjir                     18,047
Records berhasil di-join                       18,040        99.96%
Records tidak ter-join                              7         0.04%

🎯 PENCAPAIAN:
✅ Semua 18,047 records dari data banjir dipertahankan (left join)
✅ 18,040 records (100.0%) berhasil diperkaya dengan data kecamatan
✅ Menambahkan ~21 kolom baru dari data kecamatan (demografi, ekonomi, dll)

💡 Catatan:
   Records yang tidak ter-join (7) kemungkinan karena:
   - Nama kecamatan tidak ada di dataset kecamatan_filtered.csv
   - Perubahan nama kecamatan atau pemekaran wilayah
   - Perbedaan tahun data (data banjir vs data kecamatan)

📋 KOLOM-KOLOM BARU DARI DATA KECAMATAN:
    1. objectid                       (     0 values /   0.0%)
    2. kode_kec_spatial               (18,040 values / 100.0%)
    3. 